In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
!pip install pyyaml==5.1
#!pip uninstall torch
!pip uninstall detectron2
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

In [ ]:
from pathlib import Path
import os
import sys
import numpy as np

base = ...

from dataset_utils import register
from augmentation_utils import DatasetMapperAugment, OurTrainer

In [ ]:
datasets_path = base / 'datasets'
os.listdir(datasets_path / "fixed_mixup_train")

In [ ]:
register("fixed_mixup_train",
         datasets_path / "fixed_mixup_train" / "fixed_subset_labels.json",
         datasets_path / "fixed_mixup_train" / "subsets")
register("smaller_fake_australia_val",
         datasets_path / "smaller_fake_australia_val" / "labels.json",
         datasets_path / "smaller_fake_australia_val" / "data")
register("smaller_fake_australia_train",
         datasets_path / "smaller_fake_australia_train" / "labels.json",
         datasets_path / "fake_australia_train")
register("smaller_fake_bangladesh_train",
         datasets_path / "smaller_fake_bangladesh_train" / "labels.json",
         datasets_path / "smaller_fake_bangladesh_train" / "data")
register("australia_val",
         datasets_path / "australia_val" / "labels.json",
         datasets_path / "australia_val" / "data")
register("smaller_bangladesh_val",
         datasets_path / "smaller_bangladesh_val" / "labels.json",
         datasets_path / "smaller_bangladesh_val" / "data")
register("duke_train",
         datasets_path / "duke_train" / "labels.json",
         datasets_path / "duke_train" / "data")
register("smaller_duke_train",
         datasets_path / "smaller_duke_train" / "labels.json",
         datasets_path / "smaller_duke_train" / "data")
register("smaller_duke_val",
         datasets_path / "smaller_duke_val" / "labels.json",
         datasets_path / "smaller_duke_val" / "data")
register("duke_val",
         datasets_path / "duke_val" / "labels.json",
         datasets_path / "duke_val" / "data")

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.OUTPUT_DIR = str(base / "results" / ("run_"+str(np.random.randint(0, 10000))))

print(f"FIND EXPERIMENT RESULTS AT \n {cfg.OUTPUT_DIR}")

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.SOLVER.MAX_ITER = 4000
cfg.TEST.INTERVAL = 3999
cfg.SOLVER.STEPS = (4500, 5250)
cfg.SOLVER.IMS_PER_BATCH = 8

cfg.SOLVER.STRONG_AUGMENT = False

cfg.DATASETS.TRAIN = ("duke_train")
cfg.DATASETS.EVAL = ["duke_val"]
cfg.DATASETS.TEST = []

trainer = OurTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

os.rename(cfg.OUTPUT_DIR, cfg.OUTPUT_DIR + "_" + cfg.DATASETS.TRAIN + cfg.DATASETS.EVAL[-1] + f"_strong_augment_{cfg.SOLVER.STRONG_AUGMENT}_complete")